<a href="https://colab.research.google.com/github/kahramanmurat/stock-markets-analytics-zoomcamp-2024/blob/main/01-intro-and-data-sources/homework/homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Module 1 Homework

In this homework, we're going to download finance data from various sources and make simple calculations/analysis.

In [2]:
# install main library YFinance
!pip install yfinance

In [41]:
#IMPORTS
import numpy as np
import pandas as pd

#IMPORT FIN DATA SOURCES
import pandas_datareader as pdr
import yfinance as yf

#IMPORT DATA VIZ
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date, datetime


---
### Question 1: [Macro] Average growth of GDP in 2023

**What is the average growth (in %) of GDP in 2023?**

Download the timeseries Real Gross Domestic Product (GDPC1) from FRED (https://fred.stlouisfed.org/series/GDPC1).
Calculate year-over-year (YoY) growth rate (that is, divide current value to one 4 quarters ago). Find the average YoY growth in 2023 (average from 4 YoY numbers).
Round to 1 digit after the decimal point: e.g. if you get 5.66% growth => you should answer  5.7

---

In [37]:
end=date.today()
start=date(end.year-3,end.month,end.day)

In [38]:
#  Real Gross Domestic Product (GDPC1) from FRED, Billions of Chained 2017 Dollars, QUARTERLY
# https://fred.stlouisfed.org/series/GDPC1
gdpc1 = pdr.DataReader("GDPC1", "fred", start=start)

In [39]:
gdpc1['gdpc1_us_yoy'] = round((gdpc1.GDPC1/gdpc1.GDPC1.shift(4)-1)*100,1)

In [40]:
gdpc1.tail(4)

,GDPC1,gdpc1_us_yoy
DATE,,
2023-01-01,22112.329,1.7
2023-04-01,22225.350,2.4
2023-07-01,22490.692,2.9
2023-10-01,22679.255,3.1


---
### Question 2. [Macro] Inverse "Treasury Yield"

**Find the min value of (dgs10-dgs2) after since year 2000 (2000-01-01) and write it down as an answer, round to 1 digit after the decimal point.**


Download DGS2 and DGS10 interest rates series (https://fred.stlouisfed.org/series/DGS2,
 https://fred.stlouisfed.org/series/DGS10). Join them together to one dataframe on date (you might need to read about pandas.DataFrame.join()), calculate the difference dgs10-dgs2 daily.

(Additional: think about what does the "inverted yield curve" mean for the market and investors? do you see the same thing in your country/market of interest? Do you think it can be a good predictive feature for the models?)

---

In [42]:
start = datetime(2000, 1, 1)

In [54]:
# DGS2 and DGS10 interest rates series (https://fred.stlouisfed.org/series/DGS2, https://fred.stlouisfed.org/series/DGS10)
# Fetch DGS2 and DGS10 interest rate data from FRED
dgs2 = pdr.DataReader("DGS2", "fred", start=start)
dgs10 = pdr.DataReader("DGS10", "fred", start=start)

# Join the two DataFrames on the date index
yield_data = dgs10.join(dgs2, how='inner')

# Calculate the difference (DGS10 - DGS2)
yield_data['yield_diff'] = yield_data['DGS10'] - yield_data['DGS2']

# Find the row with the minimum yield difference
min_yield_row = yield_data[yield_data['yield_diff'] == yield_data['yield_diff'].min()]

# Display the result
print(round(min_yield_row, 1))

            DGS10  DGS2  yield_diff
DATE                               
2023-07-03    3.9   4.9        -1.1


# Additional:

The concept of an inverted yield curve refers to a situation in financial markets where long-term debt instruments have a lower yield than short-term debt instruments of the same credit quality. This is an unusual situation because typically, longer-term securities have higher yields to compensate investors for the greater risk of holding them over a longer period.

### What an Inverted Yield Curve Means for the Market and Investors

*   **Recession Indicator:** Historically, an inverted yield curve has been one
of the most reliable predictors of a forthcoming recession. The inversion of the yield curve has preceded many of the U.S. recessions in recent decades.

*   **Investor Sentiment:** An inverted yield curve reflects negative investor sentiment about the future of the economy. Investors may expect that the central bank will have to lower interest rates in the future to stimulate a slowing economy.

*   **Lending Impact:** It can disincentivize banks and other financial institutions from lending, as the profit margin from borrowing short-term and lending long-term diminishes, potentially slowing economic growth further.


*   In the United States, the inverted yield curve is closely watched as a recession predictor.



---
### Question 3. [Index] Which Index is better recently?

**Compare S&P 500 and IPC Mexico indexes by the 5 year growth and write down the largest value as an answer (%)**

Download on Yahoo Finance two daily index prices for S&P 500 (^GSPC, https://finance.yahoo.com/quote/%5EGSPC/) and IPC Mexico (^MXX, https://finance.yahoo.com/quote/%5EMXX/). Compare 5Y growth for both (between 2019-04-09 and 2024-04-09). Select the higher growing index and write down the growth in % (closest integer %). E.g. if ratio end/start was 2.0925 (or growth of 109.25%), you need to write down 109 as your answer.

(Additional: think of other indexes and try to download stats and compare the growth? Do create 10Y and 20Y growth stats. What is an average yearly growth rate (CAGR) for each of the indexes you select?)

---

In [65]:
# Define the tickers and the date range
tickers = ["^GSPC", "^MXX"]  # S&P 500 and IPC Mexico
start_date = "2019-04-09"
end_date = "2024-04-09"

In [66]:
# Fetch the historical data
data = yf.download(tickers, start=start_date, end=end_date)

[*********************100%%**********************]  2 of 2 completed


In [71]:
# Calculate the 5-year growth for both indices:

growth_percentages = {}

for ticker in tickers:
    start_price = data["Adj Close"][ticker].iloc[0]
    end_price = data["Adj Close"][ticker].iloc[-1]
    growth_percentage = ((end_price / start_price) - 1) * 100
    growth_percentages[ticker] = int(round(growth_percentage))

In [73]:
# Find and print the index with the highest growth:

best_index_growth = max(growth_percentages, key=growth_percentages.get)
best_growth_value = growth_percentages[best_index_growth]
print(f"The index with the highest 5-year growth is {best_index_growth} with a growth of {best_growth_value}%")

The index with the highest 5-year growth is ^GSPC with a growth of 81%


# Additional:

Let's compare with VTI and QQQ (10Y)

In [95]:
# Define the tickers and the date range
tickers = ["VTI", "ITOT"]  # Vanguard Total Stock Market ETF & IShares Core S&P Total US Stock Market ETF
start_date = "2014-04-09"
end_date = "2024-04-09"

In [96]:
# Fetch the historical data
data = yf.download(tickers, start=start_date, end=end_date)

[*********************100%%**********************]  2 of 2 completed


In [97]:
# Calculate the 10-year growth for both indices:

growth_percentages = {}

for ticker in tickers:
    start_price = data["Adj Close"][ticker].iloc[0]
    end_price = data["Adj Close"][ticker].iloc[-1]
    growth_percentage = ((end_price / start_price) - 1) * 100
    growth_percentages[ticker] = int(round(growth_percentage))

In [98]:
# Find and print the index with the highest growth:

best_index_growth = max(growth_percentages, key=growth_percentages.get)
best_growth_value = growth_percentages[best_index_growth]
print(f"The index with the highest 10-year growth is {best_index_growth} with a growth of {best_growth_value}%")

The index with the highest 10-year growth is ITOT with a growth of 219%


Let's compare with VIG and DGRO (20Y)

In [118]:
# Define the tickers and the date range
tickers = ["VTI", "^GSPC"]
start_date = "2004-04-09"
end_date = "2024-04-09"

In [119]:
# Fetch the historical data
data = yf.download(tickers, start=start_date, end=end_date)

[*********************100%%**********************]  2 of 2 completed


In [120]:
# Calculate the 20-year growth for both indices:

growth_percentages = {}

for ticker in tickers:
    start_price = data["Adj Close"][ticker].iloc[0]
    end_price = data["Adj Close"][ticker].iloc[-1]
    growth_percentage = ((end_price / start_price) - 1) * 100
    growth_percentages[ticker] = int(round(growth_percentage))

In [121]:
# Find and print the index with the highest growth:

best_index_growth = max(growth_percentages, key=growth_percentages.get)
best_growth_value = growth_percentages[best_index_growth]
print(f"The index with the highest 20-year growth is {best_index_growth} with a growth of {best_growth_value}%")

The index with the highest 20-year growth is VTI with a growth of 572%


---
### Question 4. [Stocks OHLCV] 52-weeks range ratio (2023) for the selected stocks


**Find the largest range ratio [=(max-min)/max] of Adj.Close prices in 2023**


Download the 2023 daily OHLCV data on Yahoo Finance for top5 stocks on earnings (https://companiesmarketcap.com/most-profitable-companies/): 2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM.

Here is the example data you should see in Pandas for "2222.SR": https://finance.yahoo.com/quote/2222.SR/history

Calculate maximum-minimim "Adj.Close" price for each stock and divide it by the maximum "Adj.Close" value.
Round the result to two decimal places (e.g. 0.1575 will be 0.16)

(Additional: why this may be important for your research?)

---

In [122]:
# Define the tickers for the stocks
tickers = ["2222.SR", "BRK-B", "AAPL", "MSFT", "GOOG", "JPM"]

In [123]:
# Set the time period for 2023
start_date = "2023-01-01"
end_date = "2023-12-31"

In [124]:
# Initialize a dictionary to store range ratios
range_ratios = {}

In [125]:
# Fetch the data and calculate the range ratio
for ticker in tickers:
    # Download the stock data
    stock_data = yf.download(ticker, start=start_date, end=end_date)

    # Calculate the maximum and minimum of the Adjusted Close prices
    max_price = stock_data['Adj Close'].max()
    min_price = stock_data['Adj Close'].min()

    # Calculate the range ratio
    range_ratio = (max_price - min_price) / max_price

    # Store the result, rounded to two decimal places
    range_ratios[ticker] = round(range_ratio, 2)

# Find the stock with the largest range ratio
stock_with_largest_ratio = max(range_ratios, key=range_ratios.get)
largest_ratio = range_ratios[stock_with_largest_ratio]

print("Stock with the largest range ratio:", stock_with_largest_ratio)
print("Largest range ratio:", largest_ratio)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Stock with the largest range ratio: MSFT
Largest range ratio: 0.42


---
### Question 5. [Stocks] Dividend Yield
**Find the largest dividend yield for the same set of stocks**

Use the same list of companies (2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM) and download all dividends paid in 2023.
You can use `get_actions()` method or `.dividends` field in yfinance library (https://github.com/ranaroussi/yfinance?tab=readme-ov-file#quick-start)

Sum up all dividends paid in 2023 per company and divide each value by the closing price (Adj.Close) at the last trading day of the year.

Find the maximm value in % and round to 1 digit after the decimal point. (E.g., if you obtained $1.25 dividends paid and the end year stock price is $100, the dividend yield is 1.25% -- and your answer should be equal to 1.3)

---

In [135]:
# Define the tickers for the stocks
tickers = ["2222.SR", "BRK-B", "AAPL", "MSFT", "GOOG", "JPM"]

# Set the time period for 2023
start_date = "2023-01-01"
end_date = "2023-12-31"

# Initialize a dictionary to store the dividend yields
dividend_yields = {}

# Fetch the data and calculate the dividend yield
for ticker in tickers:
    stock = yf.Ticker(ticker)

    # Get dividend actions and filter for 2023
    dividends = stock.dividends[start_date:end_date].sum()

    # Get the closing price on the last trading day of 2023
    # We adjust the date range to ensure data availability
    stock_data = yf.download(ticker, start="2023-12-20", end="2024-01-05")

    if not stock_data.empty:
        closing_price = stock_data['Adj Close'][-1]  # Get the last available closing price
        # Calculate the dividend yield
        dividend_yield = (dividends / closing_price) * 100
        # Store the result, rounded to one decimal place
        dividend_yields[ticker] = round(dividend_yield, 1)
    else:
        print(f"No data available for {ticker} near the end of 2023.")
        dividend_yields[ticker] = 0.0

# Find the stock with the highest dividend yield
if dividend_yields:
    stock_with_highest_yield = max(dividend_yields, key=dividend_yields.get)
    highest_yield = dividend_yields[stock_with_highest_yield]
    print("Stock with the highest dividend yield:", stock_with_highest_yield)
    print("Highest dividend yield:", highest_yield)
else:
    print("No valid dividend yield data available for any stock.")


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Stock with the highest dividend yield: 2222.SR
Highest dividend yield: 2.8



---
### Question 6. [Exploratory] Investigate new metrics

**Free text answer**

Download and explore a few additional metrics or time series that might be valuable for your project and write down why (briefly).

---

### **1. Beta Coefficient**
What It Is: Beta measures the volatility of a stock or portfolio in relation to the overall market. A beta greater than 1 indicates higher volatility than the market, while a beta less than 1 indicates lower volatility.
Why It’s Valuable: Understanding beta helps in assessing risk and can be crucial for portfolio diversification and risk management. It is particularly useful for determining the risk-return profile of investment opportunities.

In [137]:
def get_beta(ticker):
    stock = yf.Ticker(ticker)
    beta = stock.info.get('beta', None)  # Retrieves the Beta value from the stock's info dictionary
    return beta

# Example usage
ticker = "AAPL"  # Replace 'AAPL' with your stock ticker
beta_value = get_beta(ticker)
print(f"The Beta coefficient for {ticker} is: {beta_value}")

The Beta coefficient for AAPL is: 1.276


### **2. Earnings Per Share (EPS) Growth**
What It Is: EPS growth is the percentage change in earnings per share over time. It is a signal of a company’s profitability and financial health.
Why It’s Valuable: Tracking EPS growth can help investors identify companies with improving profitability trends, making it a key metric for stock selection in growth investing strategies.

In [154]:
!pip install beautifulsoup4

In [155]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [159]:
url = 'https://finance.yahoo.com/calendar/earnings?from=2020-07-26&to=2020-08-01&day=2020-07-27'

In [160]:
r = requests.get(url)


In [161]:
r.ok

True

In [163]:
soup = BeautifulSoup(r.text)
table = soup.find_all('table')
len(table)

1

In [164]:
#spans = soup.table.thead.find_all('span')
spans = soup.table.thead.find_all('th')
columns = []
for span in spans:
  print(span.text)
  columns.append(span.text)

Symbol
Company
Event Name
Earnings Call Time
EPS Estimate
Reported EPS
Surprise(%)
